In [1]:
import os
from openai import OpenAI
from datetime import datetime
from utils import *
import json

def log_interaction(
        response,
        messages,
        timestamp_call,
        timestamp_response,
        data_path='data/',
        log_file='log.txt'):
    
    model, tokens, cost = get_response_metadata(response)
    response_message = get_response_content(response)
    response_time = (timestamp_response - timestamp_call).total_seconds()

    timestamp_call_str = timestamp_call.strftime('%Y-%m-%d %H:%M:%S')
    timestamp_response_str = timestamp_response.strftime('%Y-%m-%d %H:%M:%S')

    metadata = {"timestamp_call": timestamp_call_str,
                "timestamp_response": timestamp_response_str,
                "input_tokens": tokens[0],
                "output_tokens": tokens[1],
                "total_tokens": sum(tokens),
                "input_message": messages,
                "response_message": response_message,
                "model": model,
                "cost ($)": f'{cost:.4f}',
                "response_time (s)": response_time,  # Example response time in seconds
            }
    
    with open(os.path.join(data_path, log_file), 'a') as file:
        json.dump(metadata, file, indent=4)
        file.write('\n')
    return

def timestamp_pretty():
    return datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [2]:
def api_conversation(
        message,
        messages=None,
        client=None,
        model='gpt-4o',
        system_prompt='You are a helpful assistant.',
        max_tokens=None,
        temperature=None):

    if not client:
        client = OpenAI()

    if not messages:
        messages = [{
            'role': 'system',
            'content': system_prompt
            }]
        
    messages.append({
        'role': 'user',
        'content': message
        })

    t0 = datetime.now()
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        n=1,
        max_tokens=max_tokens,
        response_format={'type': 'text'},
        temperature=temperature
    )
    t1 = datetime.now()

    messages.append({
        'role': 'assistant',
        'content': get_response_content(response)
        })
    
    return response, messages, t0, t1      

In [5]:
prompt = 'what are we talking about'
response, messages, t0, t1 = api_conversation(message=prompt, messages=messages)
print(get_response_content(response))  
print(get_response_tokens(response))

We are discussing the distance of Pluto from Earth and its classification in the Solar System. Specifically:

1. **Distance of Pluto from Earth**: On average, Pluto is about 3.67 billion miles (5.9 billion kilometers) away from Earth.
2. **Pluto's Classification**: Pluto was reclassified from a full-fledged planet to a "dwarf planet" in 2006 by the International Astronomical Union (IAU). The reclassification was based on the criteria set by the IAU, where Pluto did not meet the requirement of having cleared its orbital neighborhood of other debris.
(229, 123)


In [7]:
messages

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user',
  'content': 'how far away is the planet pluto (and is it technically a plsen)?'},
 {'role': 'assistant',
  'content': 'Pluto is, on average, about 3.67 billion miles (5.9 billion kilometers) away from Earth. This distance can vary significantly due to the elliptical nature of Pluto\'s orbit around the Sun.\n\nRegarding its status, Pluto is currently classified as a "dwarf planet." This reclassification happened in 2006 when the International Astronomical Union (IAU) defined the term "planet" in a way that excluded Pluto. According to the IAU, a full-fledged planet must meet three criteria:\n1. It must orbit the Sun.\n2. It must be spherical in shape due to its gravity.\n3. It must have "cleared the neighborhood" around its orbit, meaning it must be gravitationally dominant enough to clear its orbit of other debris.\n\nPluto meets the first two criteria but does not meet the third, which led to its reclas